# HaMLeT

## Session 6: Backpropagation
by Leon Weninger and Raphael Kolk

### Goal of this Session

In this session you will, step by step, implement a the backpropagation algorithm yourself without using any deep learning libraries. You should already be familiar with Python as well as NumPy (a package for scientific computing with Python).

### Given code

**Task 0:** Familiarize yourself briefly with the given code. Pay particural attention to the `Layer` and `Cost` classes, from which you will derive the classes you implement, and the `Sigmoid` layer which is predefined as an example. You'll also need to execute the cells in this section once.

The following code loads the data and trains the network in a similar fashion as in the last session.

In [7]:
import numpy as np
from tqdm import tqdm
from load_mnist import MNIST


def vectorize(j):
    label_vector = np.zeros((1, 10))
    label_vector[0, int(j)] = 1.0
    return label_vector


def load_data():
    mnist = MNIST()
    images, labels = mnist.data, mnist.target

    image_size = images.shape[1]
    label_size = labels.shape[1]

    random_permutation = np.random.permutation(images.shape[0])
    images = images[random_permutation, :]
    labels = labels[random_permutation, :]
    
    images = (images - np.mean(images))/np.std(images)

    return images, labels, image_size, label_size


def train(net, cost_function, number_epochs, batch_size, learning_rate):
    images, labels, image_size, label_size = load_data()
    training_images, validation_images = images[:50000], images[50000:]
    training_labels, validation_labels = labels[:50000], labels[50000:]

    for e in range(number_epochs):
        cost = train_epoch(e, net, training_images, training_labels, cost_function, batch_size, learning_rate)
        accuracy = validate_epoch(e, net, validation_images, validation_labels, batch_size)
        print('cost=%5.6f, accuracy=%2.6f' % (cost, accuracy), flush=True)


def train_epoch(e, net, images, labels, cost_function, batch_size, learning_rate):
    epoch_cost = 0

    for i in tqdm(range(0, len(images), batch_size), ascii=False, desc='training,   e=%i' % e):
        batch_images = images[i:min(i + batch_size, len(images)), :]
        batch_labels = labels[i:min(i + batch_size, len(labels)), :]

        # zero the gradients
        net.zero_gradients()

        # forward pass
        prediction = net.forward(batch_images)
        cost = cost_function.estimate(batch_labels, prediction)

        # backward pass
        dprediction = cost_function.gradient(cost)
        net.backward(dprediction)

        # update the parameters using the computed gradients via stochastic gradient descent.
        net.update_parameters(learning_rate)

        epoch_cost += np.mean(cost)

    return epoch_cost


def validate_epoch(e, net, images, labels, batch_size):
    n_correct = 0
    n_total = 0

    for i in tqdm(range(0, len(images), batch_size), ascii=False, desc='validation, e=%i' % e):
        batch_images = images[i:min(i + batch_size, len(images)), :]
        batch_labels = labels[i:min(i + batch_size, len(labels)), :]

        # compute predicted probabilities.
        predictions = net.forward(batch_images)

        # find the most probable class label.
        n_correct += sum(np.argmax(batch_labels, axis=1) == np.argmax(predictions, axis=1))
        n_total += batch_labels.shape[0]

    return n_correct / n_total


Remember the sigmoid function and its derivative you implemented in the previous session.

In [8]:
def sigmoid_function(var):
    return 1.0 / (1.0 + np.exp(-var))


def sigmoid_derivative(z):
    return sigmoid_function(z) * (1 - sigmoid_function(z))

The following abstract classes should serve as parent classes for all the different layers and cost functions which you will implement. 

In [9]:
class Layer:
    def __init__(self):
        # Initialize all member variables of the layer.
        pass

    def forward(self, x_in):
        # Implemets for forward pass of the layer and returns x_out.
        pass

    def backward(self, d_out):
        # Implements the backward pass of the layer and returns d_in.
        pass

    def zero_gradients(self):
        # Sets all gradients of the layer to zero.
        pass

    def update_parameters(self, learning_rate):
        # Update the parameters of the layer with the help of the gradients stored during the backward pass.
        pass


class Cost:
    def __init__(self):
        # Initialize all member variables of the cost function.
        pass

    def estimate(self, target, prediction):
        # Estimates and return the cost with respect to the predicted label and a target label previously set by set_target().
        pass

    def gradient(self, cost):
        # Calculates and returns the gradient with respect to the cost.
        pass

The following class derived from the Layer class implements the forward and backward pass of the sigmoid activation function alread known from the previous session and serves as an example for you. Since it does not have learnable parameters, no `update_parameters` or `zero_gradients` function needs to be implemented.

In [10]:
class Sigmoid(Layer):
    def __init__(self):
        self.x_in = None
    
    def forward(self, x_in):
        self.x_in = x_in
        x_out = sigmoid_function(x_in)
        return x_out
    
    def backward(self, d_out):
        d_in = d_out * sigmoid_derivative(self.x_in)
        return d_in
    
    def zero_gradients(self):
        pass
    
    def update_parameters(self, learning_rate):
        pass

### Theoretical Foundation 

**Task 1a:** Take a peace of paper and a pencil, use your knowledge from the preparation material and the introduction slides and fill in the gaps in the preparation material. Having written the formulas down, please check with the tutor if they are correct.

Before starting to work on the code, think about batched forward- and backward passing. We keep to the convention of having the batch as the first dimension of all tensors. This is consistent with modern Deep Learning Frameworks, as you will get to know in the next session. However, this convention may change when tensors needs to be transposed when performing multiplications and additions.

### Practical Implementation

**Task 2a:** Implement the `forward` function for the `Linear` layer. Remember to store the `x_in` for use in the backward pass.

**Task 2b:** Implement the `estimate` function for the `MeanSquareError` cost, which estimates the cost after a ground truth target is set. Remember to store the `prediction` for calculating the gradient.

**Task 2c:** Implement the `gradient` function for the `MeanSquareError` cost, which calculates the gradient with respect to the cost. Use the `prediction` stored during the forward pass.

**Task 2d:** Implement the `backward` function for the `Linear` layer. The function should also calculate and accumulate the gradient of `w` and `b` with regard to the error.

**Task 2e:** Implement the `update_parameters` function for the `Linear` layer, i.e., use the gradients `dw` and `db` together with a given `learning_rate` to update the parameters `w` and `b` accordingly.

**Task 2f:** Test your implementation by propagating random input through a linear layer followed by a sigmoid layer, estimating the mean square error to a random target, calculating the gradient and propagating it back through the sigmoid and linear layer. Afterwards update the parameters of the linear layer using the function you implemented.

**Task 3a:** Implement the `Network` class which can encapsulate multiple layers. It offers the same interface as a layer and is therefore derived from the `Layer` parent as well. Make sure to implement all member functions needed. The `forward` function propagates a given input through all encapsulated layers and returns the final prediction of the network, whereas the `backward` function propagates a given gradient through all layers in reversed order. `zero_gradients` and `update_parameters` invoke the respective functions of the encapsulated layers.

**Task 3b:** Test your implementation analogous to task 2f but using the `Network` class to encapsulate the linear and sigmoid layer.

**Task 4:** Train the network you just implemented using the dataloader and train function given above and the hyperparameter given below.

**Task 5:** Come up with a more sophisticated network structure and adjust the hyperparameter in order to increase the accuracy.

In [11]:
class Linear(Layer):
    def __init__(self, n_in, n_out, initial_sigma=0.1):
        self.n_in = n_in
        self.n_out = n_out

        self.w = initial_sigma * np.random.randn(n_out, n_in)
        self.b = np.zeros((1, n_out))

        self.zero_gradients()

        self.x_in = None

    def forward(self, x_in):
        # ----- Add code for task 2a between comments -----
        # -------------------------------------------------
        return x_out

    def backward(self, d_out):
        # ----- Add code for task 2d between comments -----
        # -------------------------------------------------
        return self.d_in

    def zero_gradients(self):
        self.dw = np.zeros((self.n_out, self.n_in))
        self.db = np.zeros((1, self.n_out))
        self.dx = np.empty((0, self.n_in))

    def update_parameters(self, learning_rate):
        # ----- Add code for task 2e between comments -----
        # -------------------------------------------------


class MeanSquareError(Cost):
    def __init__(self):
        self.prediction = None
        self.target = None

    def estimate(self, target, prediction):
        # ----- add code for task 2b between comments -----
        # -------------------------------------------------
        return cost

    def gradient(self, cost):
        # ----- add code for task 2c between comments -----
        # -------------------------------------------------
        return gradient


class Network(Layer):
    def __init__(self, layers):
        self.layers = layers

    # ----- add code for task 3a between comments -----
    # -------------------------------------------------

In [ ]:
# define hyperparameters
input_size = 28**2
label_size = 10
batch_size = 600
learning_rate = 0.0001
number_epochs = 100

random_input = np.random.rand(batch_size, input_size)
random_label = np.random.rand(batch_size, label_size)

linear_layer = Linear(input_size, label_size)
sigmoid_layer = Sigmoid()
cost_function = MeanSquareError()

# ----- add code for task 2f between comments -----
# -------------------------------------------------

# ----- add code for task 3b between comments -----
# -------------------------------------------------

# ----- add code for task 4 between comments ------
# -------------------------------------------------

# ----- add code for task 5 between comments ------
# -------------------------------------------------

/miniconda3/envs/py36n/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
validation, e=0: 100%|██████████| 34/34 [00:00<00:00, 134.09it/s]

cost=4.257650, accuracy=0.480400



validation, e=1: 100%|██████████| 34/34 [00:00<00:00, 189.29it/s]

cost=2.631974, accuracy=0.585800



validation, e=2: 100%|██████████| 34/34 [00:00<00:00, 204.06it/s]

...


validation, e=6: 100%|██████████| 34/34 [00:00<00:00, 149.29it/s]

cost=1.826022, accuracy=0.817200



validation, e=7: 100%|██████████| 34/34 [00:00<00:00, 150.47it/s]

cost=1.710642, accuracy=0.828200



training,   e=8:  65%|██████▌   | 55/84 [00:00<00:00, 62.66it/s]

### Feedback

Aaaaaand we're done 👏🏼🍻

If you have any suggestions on how we could improve this session, please let us know in the following cell. What did you particularly like or dislike? Did you miss any contents?

### Additional Tasks

**Task 6a:** Implement the `forward` function for the `SoftMax` layer.

**Task 6b:** Implement the `estimate` function for the `CrossEntropy` cost.

**Task 6c:** Implement the `gradient` function for the `CrossEntropy` cost.

**Task 6d:** Implement the `backward` function for the `SoftMax` layer.

**Task 6e:** Test your implementation by setting up a network using the soft max layer and cross entropy cost in combination.

In [ ]:
class SoftMax(Layer):
    def __init__(self):
        self.x_out = None

    def forward(self, x_in):
        # ----- add code for task 6a between comments -----
        # -------------------------------------------------
        return self.x_out

    def backward(self, d_out):
        # ----- add code for task 6d between comments -----
        # -------------------------------------------------
        return d_in


class CrossEntropy(Cost):
    def __init__(self):
        self.x_in = None
        self.target = None
        self.eps = 1e-12

    def estimate(self, target, x_in):
        # ----- add code for task 6b between comments -----
        # -------------------------------------------------
        return cost

    def gradient(self, d_out):
        # ----- add code for task 6c between comments -----
        # -------------------------------------------------
        return gradient


In [ ]:
# define hyperparameters
input_size = 28**2
label_size = 10
batch_size = 600
learning_rate = 0.00001
number_epochs = 100

# ----- add code for task 6e between comments -----
# -------------------------------------------------